In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    PROMPT_KGE_DIMENSION,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import (
    PromptBertClassifier,
    VanillaBertClassifier,
    InputEmbedsReplaceClassifier,
)

In [2]:
kg_manager = MovieLensManager()

In [3]:
graph_representation_generator_prompt = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    kge_dimension=PROMPT_KGE_DIMENSION,
)
graph_representation_generator_input_embeds_replace = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
)

loading pretrained model
Device: 'cpu'
loading pretrained model
Device: 'cpu'


In [4]:
prompt_embeddings = graph_representation_generator_prompt.get_saved_embeddings("prompt")
input_embeds_replace_embeddings = (
    graph_representation_generator_input_embeds_replace.get_saved_embeddings(
        "input_embeds_replace"
    )
)
save_prompt = False
save_input_embeds_replace = False
if prompt_embeddings is None:
    prompt_embeddings = graph_representation_generator_prompt.generate_embeddings(
        kg_manager.llm_df
    )
    save_prompt = True
if input_embeds_replace_embeddings is None:
    input_embeds_replace_embeddings = (
        graph_representation_generator_input_embeds_replace.generate_embeddings(
            kg_manager.llm_df
        )
    )
    save_input_embeds_replace = True

kg_manager.append_prompt_graph_embeddings(prompt_embeddings, save=save_prompt)
kg_manager.append_input_embeds_replace_graph_embeddings(
    input_embeds_replace_embeddings, save=save_input_embeds_replace
)


In [5]:
VANILLA_ROOT = f"{ROOT}/llm/vanilla"
PROMPT_ROOT = f"{ROOT}/llm/prompt"
INPUT_EMBEDS_REPLACE_ROOT = f"{ROOT}/llm/input_embeds_replace"

In [6]:
vanilla_bert_classifier = VanillaBertClassifier(
    kg_manager.llm_df,
    kg_manager.source_df,
    kg_manager.target_df,
    root_path=VANILLA_ROOT,
    false_ratio=-1,
)
prompt_bert_classifier = PromptBertClassifier(
    kg_manager,
    graph_representation_generator_prompt.get_embedding,
    root_path=PROMPT_ROOT,
    model_max_length=512,
    false_ratio=-1,
)
input_embeds_replace_bert_classifier = InputEmbedsReplaceClassifier(
    kg_manager,
    graph_representation_generator_input_embeds_replace.get_embedding,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
    false_ratio=-1,
)


Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of InputEmbedsReplaceBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
dataset_vanilla = kg_manager.generate_vanilla_dataset(
    vanilla_bert_classifier.tokenize_function
)
dataset_prompt = kg_manager.generate_prompt_embedding_dataset(
    prompt_bert_classifier.tokenize_function,
)
dataset_input_embeds_replace = (
    kg_manager.generate_input_embeds_replace_embedding_dataset(
        input_embeds_replace_bert_classifier.tokenizer.sep_token,
        input_embeds_replace_bert_classifier.tokenizer.pad_token,
        input_embeds_replace_bert_classifier.tokenize_function,
    )
)

In [9]:
BATCH_SIZE = 64
vanilla_bert_classifier.forward_dataset_and_save_outputs(
    dataset_vanilla,
    kg_manager.get_vanilla_tokens_as_df,
    force_recompute=True,
    batch_size=BATCH_SIZE,
    load_fields=["hidden_states", "logits", "attentions"],
    is_test=True,
    splits=["val"],
)
prompt_bert_classifier.forward_dataset_and_save_outputs(
    dataset_prompt,
    kg_manager.get_prompt_tokens_as_df,
    force_recompute=True,
    load_fields=["hidden_states", "logits", "attentions"],
    batch_size=BATCH_SIZE,
    is_test=True,
    splits=["val"],
)

input_embeds_replace_bert_classifier.forward_dataset_and_save_outputs(
    dataset_input_embeds_replace,
    kg_manager.get_vanilla_tokens_as_df,
    force_recompute=True,
    load_fields=["hidden_states", "logits", "attentions"],
    batch_size=BATCH_SIZE,
    is_test=True,
    splits=["val"],
)

Forward val where ('cls',) are masked
Forward val where ('user_id',) are masked
Forward val where ('movie_id',) are masked
Forward val where ('title',) are masked
Forward val where ('genres',) are masked
Forward val where ('seps',) are masked
Forward val where ('cls', 'user_id') are masked
Forward val where ('cls', 'movie_id') are masked
Forward val where ('cls', 'title') are masked
Forward val where ('cls', 'genres') are masked
Forward val where ('cls', 'seps') are masked
Forward val where ('user_id', 'movie_id') are masked
Forward val where ('user_id', 'title') are masked
Forward val where ('user_id', 'genres') are masked
Forward val where ('user_id', 'seps') are masked
Forward val where ('movie_id', 'title') are masked
Forward val where ('movie_id', 'genres') are masked
Forward val where ('movie_id', 'seps') are masked
Forward val where ('title', 'genres') are masked
Forward val where ('title', 'seps') are masked
Forward val where ('genres', 'seps') are masked
Forward val where ('cl